In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from datetime import datetime
from sklearn import preprocessing
import os
from sklearn.metrics import *
%matplotlib inline

In [2]:
from scikeras.wrappers import KerasClassifier

In [3]:
from keras.models import Sequential,load_model
from keras.layers import Dense, LSTM, BatchNormalization, Dropout, Input, TimeDistributed, RepeatVector, GRU, InputLayer
from tensorflow.keras.optimizers import Adam 
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
#from scikeras.wrappers.KerasClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from time import time
from sklearn.preprocessing import MinMaxScaler  # MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.callbacks import Callback,ModelCheckpoint
#from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
from imblearn.over_sampling import SMOTE
import os
from keras import regularizers
from keras.models import Model

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
df = pd.read_csv('./melting_tank.csv',
                 parse_dates=['STD_DT'],
                 infer_datetime_format=True,
                 index_col='STD_DT'
                )
df.drop(['NUM'],axis=1,inplace=True) # NUM column 제거
print(f'data shape = {df.shape}')
df.head(20)

data shape = (835200, 5)


,MELT_TEMP,MOTORSPEED,MELT_WEIGHT,INSP,TAG
STD_DT,,,,,
2020-03-04 00:00:00,489,116,631,3.19,OK
2020-03-04 00:00:00,433,78,609,3.19,OK
2020-03-04 00:00:00,464,154,608,3.19,OK
2020-03-04 00:00:00,379,212,606,3.19,OK
2020-03-04 00:00:00,798,1736,604,3.21,OK
2020-03-04 00:00:00,743,1722,603,3.21,OK
2020-03-04 00:00:00,390,212,602,3.19,OK
2020-03-04 00:00:00,493,152,600,3.19,OK
2020-03-04 00:00:00,427,0,599,3.19,OK


In [6]:
encoder = preprocessing.LabelEncoder()
df['TAG'] = encoder.fit_transform(df['TAG']) # categorical variable to numerical variable
df['TAG'] = df['TAG'].astype('float32')

In [7]:
split_date = int(df.shape[0]*0.7)
train = df[:split_date]
test = df[split_date:]

In [8]:
scaler = MinMaxScaler()  # MinMaxScaler(feature_range=(-1,1))
train_sc = pd.DataFrame(scaler.fit_transform(train), index=train.index, columns=train.columns)
test_sc = pd.DataFrame(scaler.transform(test), index=test.index, columns=test.columns)

In [9]:
X_train = train_sc.drop(['TAG'],axis=1)
y_train = train_sc['TAG']

In [10]:
# Class Imbalance 처리
#smote = SMOTE(random_state=0)
#X_train, y_train = smote.fit_resample(X_train.values,y_train.values)

In [11]:
# Raw data -> Sequence로 만들기
def make_dataset(data, label, window_size):
    feature_list = []
    label_list = []
    for i in range(len(data)-window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [12]:
X_train = pd.DataFrame(X_train, columns=['MELT_TEMP', 'MOTORSPEED', 'MELT_WEIGHT', 'INSP'])
y_train = pd.DataFrame(y_train, columns=['TAG'])
train_feature, train_label = make_dataset(X_train, y_train, 10)

In [13]:
X_test = test_sc.drop(['TAG'],axis=1)
y_test = test_sc['TAG']
test_feature, test_label = make_dataset(X_test, y_test, 10)

In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size = 0.3)

In [15]:
X_train.shape, y_train.shape

((409241, 10, 4), (409241, 1))

In [16]:
# F1-Score Metric으로 사용하기위한 함수
def get_f1(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [42]:
model = Sequential()

model.add(LSTM(50, input_shape=(train_feature.shape[1], train_feature.shape[2]),
                    activation='tanh', return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

In [43]:
model.fit(X_train,y_train,epochs=30,batch_size=50,callbacks=[early_stop])

Epoch 1/30
8185/8185 [==============================] - 64s 8ms/step - loss: 0.5020
Epoch 2/30
8185/8185 [==============================] - 62s 8ms/step - loss: 0.4805
Epoch 3/30
8185/8185 [==============================] - 63s 8ms/step - loss: 0.4786
Epoch 4/30
8185/8185 [==============================] - 63s 8ms/step - loss: 0.4776
Epoch 5/30
8185/8185 [==============================] - 63s 8ms/step - loss: 0.4766
Epoch 6/30
8185/8185 [==============================] - 62s 8ms/step - loss: 0.4754
Epoch 7/30
8185/8185 [==============================] - 62s 8ms/step - loss: 0.4742
Epoch 8/30
8185/8185 [==============================] - 63s 8ms/step - loss: 0.4723
Epoch 9/30
8185/8185 [==============================] - 62s 8ms/step - loss: 0.4714
Epoch 10/30
8185/8185 [==============================] - 62s 8ms/step - loss: 0.4711
Epoch 11/30
8185/8185 [==============================] - 63s 8ms/step - loss: 0.4706
Epoch 12/30
8185/8185 [==============================] - 62s 8ms/step - lo

In [33]:

# Define Early Stopping:
early_stop = EarlyStopping(monitor='val_accuracy', min_delta=0, 
                           patience=30, verbose=1, mode='auto',
                           baseline=0, restore_best_weights=True)

In [34]:
# adaboost
predictor = KerasClassifier(build_fn=lambda:model, epochs=30, batch_size=50,#validation_data=(X_valid, y_valid), 
                          callbacks=[early_stop])
final_model = AdaBoostClassifier(predictor, n_estimators=5, random_state=42)

In [35]:
history = final_model.fit(X_train, y_train)

/home/jhpark/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/_utils.py:28: UserWarning: Setting the random state for TF involves irreversibly re-setting the random seed. This may have unintended side effects.
  "Setting the random state for TF involves "
/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/_utils.py:28: UserWarning: Setting the random state for TF involves irreversibly re-setting the random se

Epoch 1/30
8185/8185 [==============================] - 61s 7ms/step - loss: 1.4776e-06
Epoch 2/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.3409e-06
Epoch 3/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.2525e-06
Epoch 4/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.2259e-06
Epoch 5/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.1948e-06
Epoch 6/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.1876e-06
Epoch 7/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.1834e-06
Epoch 8/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.1809e-06
Epoch 9/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.1791e-06
Epoch 10/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.1773e-06
Epoch 11/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.1766e-06
Epoch 12/30
8185/8185 [=======

/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/_utils.py:28: UserWarning: Setting the random state for TF involves irreversibly re-setting the random seed. This may have unintended side effects.
  "Setting the random state for TF involves "
/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/_utils.py:28: UserWarning: Setting the random state for TF involves irreversibly re-setting the random seed. This may have unintended side effects.
  "Setting the random state for TF involves "


Epoch 1/30
8185/8185 [==============================] - 60s 7ms/step - loss: 1.7043e-06
Epoch 2/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6912e-06
Epoch 3/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6898e-06
Epoch 4/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6898e-06
Epoch 5/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6895e-06
Epoch 6/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6892e-06
Epoch 7/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6892e-06
Epoch 8/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6891e-06
Epoch 9/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6891e-06
Epoch 10/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6891e-06
Epoch 11/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6888e-06
Epoch 12/30
8185/8185 [=======

/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/_utils.py:28: UserWarning: Setting the random state for TF involves irreversibly re-setting the random seed. This may have unintended side effects.
  "Setting the random state for TF involves "
/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/_utils.py:28: UserWarning: Setting the random state for TF involves irreversibly re-setting the random seed. This may have unintended side effects.
  "Setting the random state for TF involves "


Epoch 1/30
8185/8185 [==============================] - 60s 7ms/step - loss: 1.6935e-06
Epoch 2/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6934e-06
Epoch 3/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6935e-06
Epoch 4/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6935e-06
Epoch 5/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6933e-06
Epoch 6/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6933e-06
Epoch 7/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6934e-06
Epoch 8/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6934e-06
Epoch 9/30
8185/8185 [==============================] - 60s 7ms/step - loss: 1.6933e-06
Epoch 10/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6934e-06
Epoch 11/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6932e-06
Epoch 12/30
8185/8185 [=======

/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/_utils.py:28: UserWarning: Setting the random state for TF involves irreversibly re-setting the random seed. This may have unintended side effects.
  "Setting the random state for TF involves "
/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/_utils.py:28: UserWarning: Setting the random state for TF involves irreversibly re-setting the random seed. This may have unintended side effects.
  "Setting the random state for TF involves "


Epoch 1/30
8185/8185 [==============================] - 60s 7ms/step - loss: 1.6930e-06
Epoch 2/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6928e-06
Epoch 3/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6928e-06
Epoch 4/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6927e-06
Epoch 5/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6926e-06
Epoch 6/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6927e-06
Epoch 7/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6925e-06
Epoch 8/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6926e-06
Epoch 9/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6926e-06
Epoch 10/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6926e-06
Epoch 11/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6924e-06
Epoch 12/30
8185/8185 [=======

/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/_utils.py:28: UserWarning: Setting the random state for TF involves irreversibly re-setting the random seed. This may have unintended side effects.
  "Setting the random state for TF involves "
/home/jhpark/anaconda3/envs/graph/lib/python3.6/site-packages/scikeras/_utils.py:28: UserWarning: Setting the random state for TF involves irreversibly re-setting the random seed. This may have unintended side effects.
  "Setting the random state for TF involves "


Epoch 1/30
8185/8185 [==============================] - 60s 7ms/step - loss: 1.6935e-06
Epoch 2/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6934e-06
Epoch 3/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6934e-06
Epoch 4/30
8185/8185 [==============================] - 60s 7ms/step - loss: 1.6930e-06
Epoch 5/30
8185/8185 [==============================] - 59s 7ms/step - loss: 1.6931e-06
Epoch 6/30
8185/8185 [==============================] - 61s 7ms/step - loss: 1.6929e-06
Epoch 7/30
8185/8185 [==============================] - 63s 8ms/step - loss: 1.6927e-06
Epoch 8/30
8185/8185 [==============================] - 62s 8ms/step - loss: 1.6927e-06
Epoch 9/30
8185/8185 [==============================] - 62s 8ms/step - loss: 1.6924e-06
Epoch 10/30
8185/8185 [==============================] - 63s 8ms/step - loss: 1.6923e-06
Epoch 11/30
8185/8185 [==============================] - 62s 8ms/step - loss: 1.6920e-06
Epoch 12/30
8185/8185 [=======

In [36]:
pred = model.predict(test_feature)

In [37]:
pred_df = pd.DataFrame(pred, columns=['TAG'])

In [38]:
pred_df['TAG'] = pred_df['TAG'].apply(lambda x:1 if x>= 0.5 else 0)

In [39]:
p = precision_score(test_label, pred_df)
r = recall_score(test_label, pred_df)
f1 = f1_score(test_label, pred_df)
acc = accuracy_score(test_label, pred_df)

In [40]:
confusion_matrix(test_label, pred_df)

array([[  2926,     14],
       [217055,  30555]])

In [41]:
print("Precision: %0.4f" %p)
print("Recall: %0.4f" %r)
print("F1-score: %0.4f" %f1)
print("Accuracy: %0.4f" %acc)

Precision: 0.9995
Recall: 0.1234
F1-score: 0.2197
Accuracy: 0.1336


In [ ]:
for i in range(len(pred_df)):
    print(pred_df['TAG'][i],int(test_label[i]), sep=' ')